In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm_notebook
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns             

sns.set_style("darkgrid")

PATH ='/home/antixk/Anand/' #'/home/robot/Anand/'
sys.path.append(PATH)
print(sys.path)

from NeuralBlocks.models import LinearResNet
from NeuralBlocks.trainers import SupervisedTrainer

['/home/antixk/Anand/NeuralBlocks/notebooks', '/home/antixk/miniconda3/envs/main/lib/python37.zip', '/home/antixk/miniconda3/envs/main/lib/python3.7', '/home/antixk/miniconda3/envs/main/lib/python3.7/lib-dynload', '', '/home/antixk/.local/lib/python3.7/site-packages', '/home/antixk/miniconda3/envs/main/lib/python3.7/site-packages', '/home/antixk/miniconda3/envs/main/lib/python3.7/site-packages/inplace_abn-1.0.3-py3.7-linux-x86_64.egg', '/home/antixk/miniconda3/envs/main/lib/python3.7/site-packages/IPython/extensions', '/home/antixk/.ipython', '/home/antixk/Anand/']


In [2]:
torch.manual_seed(2456)
# cudnn.deterministic = True
cudnn.benchmark = True #False
np.random.seed(2456)

NUM_EPOCH = 200
BATCH_SIZE = 128
CHECKPOINT_INTERVAL = 100
LRS = [0.0001, 0.001, 0.01]
NORMS =[None,'BN', 'SN', 'WN','MWN', 'MSN', 'MSNTReLU', 'MWNTReLU']
DATA_PATH = PATH+"NeuralBlocks/data_utils/datasets/CIFAR10/"
SAVE_PATH = PATH+"NeuralBlocks/experiments/MNIST_linres/"

os.makedirs(SAVE_PATH, exist_ok=True)

In [3]:
# transform_train = transforms.Compose(
#     [#transforms.RandomAffine(degrees=7, translate=(0.1, 0.1), scale=(0.95, 1.05)), 
#      transforms.ToTensor(),
#      transforms.Normalize((0.19036,), (0.34743,)),
#     ])

# transform_test = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.19036,), (0.34743,)),
#     ])

# trainset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, download=True, transform=transform_train)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

# testset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, download=True, transform=transform_test)
# testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
x,y = next(a for a in trainloader)

In [6]:
x.size()

torch.Size([128, 3, 32, 32])

In [7]:
net =  LinearResNet(in_channels=3*32*32, num_class=10, widen_factor=10, norm='MSN').cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
trainer = SupervisedTrainer(net, data_bunch=[trainloader, testloader], optimizer=optimizer,
                           loss_function=criterion, metrics=['accuracy','top_3_accuracy'], use_cuda=True)

In [8]:
trainer.run(10)

Epoch,Train_loss,Train_accuracy,Train_top_3_accuracy,Test_loss,Test_accuracy,Test_top_3_accuracy


IndexError: list index out of range

In [ ]:
trainer.save_log(SAVE_PATH, title="MSN_model")

In [ ]:
prof.export_chrome_trace(SAVE_PATH+'log_BN.txt')